# 03 – Normalización y selección de genes altamente variables (HVG)


Notebook centrado en dos pasos sobre el objeto tras QC:
- Normalizar expresión por célula (library-size + log1p).
- Marcar genes altamente variables (HVG) para análisis posteriores.
No hace QC ni integración; solo deja el objeto listo para PCA/Harmony/UMAP.

## 1. Contexto: ¿qué es la normalización y qué son los HVG?

- Partimos de `TFM_CIRRHOSIS_afterQC.h5ad` con células ya filtradas por QC.
- Normalización: `normalize_total(target_sum=1e4)` + `log1p` para comparar células en la misma escala.
- HVG: genes con alta variabilidad entre células (más informativos), usados para PCA, Harmony, vecinos y clustering.
- Los HVG se marcan en `adata.var["highly_variable"]`; no se modifican las cuentas crudas.

In [1]:
from pathlib import Path
import json

import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd

print("Scanpy:", sc.__version__)
print("AnnData:", ad.__version__)

Scanpy: 1.11.5
AnnData: 0.12.6


C:\Users\Coni\AppData\Local\Temp\ipykernel_3320\1154666483.py:9: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('scanpy')` instead
  print("Scanpy:", sc.__version__)
C:\Users\Coni\AppData\Local\Temp\ipykernel_3320\1154666483.py:10: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  print("AnnData:", ad.__version__)


## 2. Organización de carpetas y configuración básica

- Se asume este notebook en `notebooks/` y se definen:
  - `PROJECT_ROOT`, `AI_PACKAGE_DIR`, `DATA_RAW_DIR`, `DATA_PROCESSED_DIR`.
- Se lee `AI_Package/MANIFEST.json` (si existe) para recuperar:
  - claves estándar: `EMB_KEY`, `NBR_KEY`, `UMAP_KEY`.
- `CONFIG` contiene:
  - rutas principales,
  - la ruta de entrada: `QC_H5AD_PATH = data_processed/TFM_CIRRHOSIS_afterQC.h5ad`.

In [2]:
# Detectamos la carpeta raíz del proyecto asumiendo que este notebook está en "notebooks/"
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent

AI_PACKAGE_DIR      = PROJECT_ROOT / "AI_Package"
DATA_RAW_DIR        = PROJECT_ROOT / "data_raw"
DATA_PROCESSED_DIR  = PROJECT_ROOT / "data_processed"
DATA_PROCESSED_DIR.mkdir(exist_ok=True)

print("Directorio del notebook:", NOTEBOOK_DIR)
print("Raíz del proyecto:", PROJECT_ROOT)
print("Carpeta AI_Package:", AI_PACKAGE_DIR)
print("Carpeta data_raw:", DATA_RAW_DIR)
print("Carpeta data_processed:", DATA_PROCESSED_DIR)

# Leemos MANIFEST.json si existe para recuperar claves estándar
manifest_path = AI_PACKAGE_DIR / "MANIFEST.json"
if manifest_path.exists():
    with open(manifest_path, "r", encoding="utf-8") as f:
        manifest = json.load(f)
else:
    manifest = {}
    print("\n[AVISO] No se ha encontrado MANIFEST.json; se usarán valores por defecto.")

CONFIG = {
    "PROJECT_ROOT": PROJECT_ROOT,
    "AI_PACKAGE_DIR": AI_PACKAGE_DIR,
    "DATA_RAW_DIR": DATA_RAW_DIR,
    "DATA_PROCESSED_DIR": DATA_PROCESSED_DIR,
    "QC_H5AD_PATH": DATA_PROCESSED_DIR / "TFM_CIRRHOSIS_afterQC.h5ad",
    "EMB_KEY": manifest.get("keys", {}).get("EMB_KEY", "X_pca_harmony"),
    "NBR_KEY": manifest.get("keys", {}).get("NBR_KEY", "harmony"),
    "UMAP_KEY": manifest.get("keys", {}).get("UMAP_KEY", "X_umap_harmony"),
}

CONFIG

Directorio del notebook: D:\Users\Coni\Documents\TFM_CirrhosIS\notebooks
Raíz del proyecto: D:\Users\Coni\Documents\TFM_CirrhosIS
Carpeta AI_Package: D:\Users\Coni\Documents\TFM_CirrhosIS\AI_Package
Carpeta data_raw: D:\Users\Coni\Documents\TFM_CirrhosIS\data_raw
Carpeta data_processed: D:\Users\Coni\Documents\TFM_CirrhosIS\data_processed


{'PROJECT_ROOT': WindowsPath('D:/Users/Coni/Documents/TFM_CirrhosIS'),
 'AI_PACKAGE_DIR': WindowsPath('D:/Users/Coni/Documents/TFM_CirrhosIS/AI_Package'),
 'DATA_RAW_DIR': WindowsPath('D:/Users/Coni/Documents/TFM_CirrhosIS/data_raw'),
 'DATA_PROCESSED_DIR': WindowsPath('D:/Users/Coni/Documents/TFM_CirrhosIS/data_processed'),
 'QC_H5AD_PATH': WindowsPath('D:/Users/Coni/Documents/TFM_CirrhosIS/data_processed/TFM_CIRRHOSIS_afterQC.h5ad'),
 'EMB_KEY': 'X_pca_harmony',
 'NBR_KEY': 'harmony',
 'UMAP_KEY': 'X_umap_harmony'}

## 3. Carga del objeto filtrado tras QC

- Se comprueba que `QC_H5AD_PATH` existe.
- Se carga el objeto con `sc.read_h5ad`, obteniendo `adata`.
- Estructura esperada:
  - `adata.X`: matriz de expresión,
  - `adata.obs`: metadatos por célula (incluye `gem_id`, `patientID`, etc.),
  - `adata.var`: metadatos por gen.

In [3]:
qc_h5ad_path = CONFIG["QC_H5AD_PATH"]

if not qc_h5ad_path.exists():
    raise FileNotFoundError(
        f"No se encuentra el archivo filtrado tras QC en:\n{qc_h5ad_path}\n"
        "Asegúrate de que has ejecutado el QC o ajusta CONFIG['QC_H5AD_PATH']."
    )

adata = sc.read_h5ad(qc_h5ad_path)
print(adata)

AnnData object with n_obs × n_vars = 225944 × 38606
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'gem_id', 'patientID', 'age', 'sex', 'diagnostic', 'disease', 'disease_classification', 'disease_status', 'disease_grade', 'alternative_classification', 'comorbidity', 'sample_collection', 'scrublet_doublet_scores', 'scrublet_predicted_doublet', 'total_counts_from_X', 'n_genes_from_X', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt'
    var: 'features', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'


## 4. Copia de seguridad de cuentas crudas en una capa

- Antes de normalizar, se guarda la matriz original de `adata.X` en:
  - `adata.layers["counts"]`, solo si no existe ya.
- Así:
  - `counts` conserva las cuentas crudas,
  - `adata.X` puede reutilizarse para versiones normalizadas/log1p sin perder el original.

In [4]:
# Creamos una copia de seguridad de la matriz original en adata.layers["counts"] si no existe
if "counts" not in adata.layers:
    X = adata.X
    if hasattr(X, "copy"):
        adata.layers["counts"] = X.copy()
    else:
        adata.layers["counts"] = np.array(X, copy=True)
    print("Capa 'counts' creada a partir de adata.X.")
else:
    print("Capa 'counts' ya existe; no se sobrescribe.")

Capa 'counts' creada a partir de adata.X.


## 5. Normalización por tamaño de biblioteca y log-transformación

- Normalización por célula:
  - `sc.pp.normalize_total(adata, target_sum=1e4)`.
- Transformación logarítmica:
  - `sc.pp.log1p(adata)`.
- Se guarda la matriz resultante en:
  - `adata.X` (uso directo),
  - `adata.layers["log1p_10k"]` (copia explícita del log1p-normalizado).
- `adata.layers["counts"]` sigue conteniendo las cuentas crudas para DE u otros métodos que las requieran.

In [5]:
# Normalización por tamaño de biblioteca (1e4 cuentas por célula)
sc.pp.normalize_total(adata, target_sum=1e4, inplace=True)

# Transformación log1p
sc.pp.log1p(adata)

# Guardamos el resultado en una capa dedicada
adata.layers["log1p_10k"] = adata.X.copy()

print("Normalización y log1p completadas.")
print("Capa 'log1p_10k' creada.")

Normalización y log1p completadas.
Capa 'log1p_10k' creada.


## 6. Selección de genes altamente variables (HVG)

- Se detecta automáticamente una columna de batch en `adata.obs`:
  - prioridad: `gem_id` → `libraryID` → `sample` → `sample_id` → `batch` → `patientID`.
  - Si no se encuentra ninguna con >1 categoría, no se usa `batch_key`.
- Selección de HVG:
  - `N_TOP_HVGS = 3000`.
  - `sc.pp.highly_variable_genes` con:
    - `flavor="seurat_v3"`,
    - `layer="counts"` (se usan cuentas crudas),
    - `batch_key=batch_key` si está disponible.
- Exclusión manual de genes no deseados como HVG:
  - mitocondriales (`MT-` / `var["mt"]`),
  - ribosomales (`RPL*`, `RPS*`),
  - eritroides (`HB*`),
  - HLA (`HLA-*`).
- Estos genes se marcan con `highly_variable = False` aunque el algoritmo los hubiera seleccionado.
- Resultado final:
  - `adata.var["highly_variable"]` indica qué genes se usarán como HVG en los siguientes notebooks.

In [6]:
# Detección y elección de batch_key para HVG / integración

candidate_batch_keys = ["gem_id", "libraryID", "sample", "sample_id", "batch", "donor", "patientID"]
available_obs = list(adata.obs.columns)

print("Columnas disponibles en adata.obs:")
print(available_obs)

batch_key = None

# Preferimos gem_id si existe y hay >1 biblioteca
if "gem_id" in adata.obs.columns and adata.obs["gem_id"].nunique() > 1:
    batch_key = "gem_id"
# Si no hubiera gem_id, podríamos caer a otros candidatos
elif "libraryID" in adata.obs.columns and adata.obs["libraryID"].nunique() > 1:
    batch_key = "libraryID"
elif "sample" in adata.obs.columns and adata.obs["sample"].nunique() > 1:
    batch_key = "sample"
elif "sample_id" in adata.obs.columns and adata.obs["sample_id"].nunique() > 1:
    batch_key = "sample_id"
elif "batch" in adata.obs.columns and adata.obs["batch"].nunique() > 1:
    batch_key = "batch"
# patientID lo dejamos como última opción, solo si se quisiera tratar como batch
elif "patientID" in adata.obs.columns and adata.obs["patientID"].nunique() > 1:
    batch_key = "patientID"

print("\nBatch key detectado automáticamente:", batch_key)

if batch_key is not None:
    print(f"Valores únicos en '{batch_key}': {adata.obs[batch_key].nunique()}")
    print(adata.obs[batch_key].value_counts().head())
else:
    print("No se utilizará batch_key en la selección de HVG.")

Columnas disponibles en adata.obs:
['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'gem_id', 'patientID', 'age', 'sex', 'diagnostic', 'disease', 'disease_classification', 'disease_status', 'disease_grade', 'alternative_classification', 'comorbidity', 'sample_collection', 'scrublet_doublet_scores', 'scrublet_predicted_doublet', 'total_counts_from_X', 'n_genes_from_X', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt']

Batch key detectado automáticamente: gem_id
Valores únicos en 'gem_id': 8
gem_id
ee31rg5x_hsm6kstq    38633
w1oxhhll_d4comu1j    37787
eoogcieu_var4q976    37766
hner8v5o_m8skafb5    35404
glol8w2k_5hkzluhi    23573
Name: count, dtype: int64


In [7]:
pip install --user scikit-misc

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Número de HVGs a seleccionar (ajustable)
N_TOP_HVGS = 3000

print(f"Seleccionando hasta {N_TOP_HVGS} genes altamente variables...")

# Nos aseguramos de que existe la capa 'counts'
if "counts" not in adata.layers:
    raise RuntimeError("La capa 'counts' no está disponible. Debe contener las cuentas crudas.")

hvg_kwargs = dict(
    flavor="seurat_v3",
    n_top_genes=N_TOP_HVGS,
    layer="counts",      # usamos las cuentas crudas, no la matriz normalizada
    inplace=True
)

if batch_key is not None:
    hvg_kwargs["batch_key"] = batch_key

sc.pp.highly_variable_genes(adata, **hvg_kwargs)

print("Columnas HVG añadidas a adata.var:")
print([col for col in adata.var.columns if "highly_variable" in col])

Seleccionando hasta 3000 genes altamente variables...
Columnas HVG añadidas a adata.var:
['highly_variable', 'highly_variable_rank', 'highly_variable_nbatches']


In [9]:
# --- Exclusión de genes no deseados como HVG (MT / ribosomales / HB / HLA) ---

var = adata.var
gene_names = var.index.to_series().astype(str).str.upper()

# Usamos la columna 'mt' si existe; si no, la reconstruimos
if "mt" in var.columns:
    mt_genes = var["mt"].astype(bool)
else:
    mt_genes = gene_names.str.startswith("MT-")

rpl_genes = gene_names.str.startswith("RPL")
rps_genes = gene_names.str.startswith("RPS")
hb_genes  = gene_names.str.startswith("HB")      # incluye HBA, HBB, etc.
hla_genes = gene_names.str.startswith("HLA-")    # HLA de clase I/II

exclude_mask = mt_genes | rpl_genes | rps_genes | hb_genes | hla_genes

n_hvgs_before = int(var["highly_variable"].sum())
adata.var.loc[exclude_mask, "highly_variable"] = False
n_hvgs_after = int(adata.var["highly_variable"].sum())

print(f"HVG antes de excluir MT/Ribo/HB/HLA: {n_hvgs_before}")
print(f"HVG después de excluir MT/Ribo/HB/HLA: {n_hvgs_after}")
print(f"Genes excluidos de HVG por ser MT/Ribo/HB/HLA: {int(exclude_mask.sum())}")

# Número total de genes y número de HVG finales
n_genes_total = adata.n_vars
n_hvgs = int(adata.var["highly_variable"].sum())

print(f"\nNúmero total de genes: {n_genes_total}")
print(f"Número de genes altamente variables (final): {n_hvgs}")

# Mostrar los primeros HVG (por conveniencia)
hvgs = adata.var.index[adata.var["highly_variable"]].tolist()
print("\nPrimeros 20 genes altamente variables:")
print(hvgs[:20])

HVG antes de excluir MT/Ribo/HB/HLA: 3000
HVG después de excluir MT/Ribo/HB/HLA: 2976
Genes excluidos de HVG por ser MT/Ribo/HB/HLA: 157

Número total de genes: 38606
Número de genes altamente variables (final): 2976

Primeros 20 genes altamente variables:
['ISG15', 'TP73', 'SMIM1', 'ENSG00000236948', 'ICMT-DT', 'SLC2A7', 'SPSB1', 'RBP7', 'TNFRSF1B', 'DHRS3', 'KAZN', 'TMEM51', 'CELA2A', 'MFAP2', 'PADI4', 'CDA', 'ENSG00000236936', 'ALPL', 'RAP1GAP', 'C1QA']


## 7. Creación del objeto restringido a HVG

- Conceptualmente, los análisis posteriores (PCA, Harmony, vecinos/UMAP) se realizarán:
  - usando **todas las células**,
  - pero solo genes con `adata.var["highly_variable"] == True`.
- En este notebook **no se crea un AnnData separado solo con HVG**;
  el subconjunto HVG se aplicará más adelante mediante subsetting.

## 8. Guardado de salidas normalizadas

- Se guarda un único objeto normalizado, con todas las células y genes:

  - `data_processed/TFM_CIRRHOSIS_normalized.h5ad`

- Contiene:
  - `layers["counts"]`: cuentas crudas,
  - `adata.X` y `layers["log1p_10k"]`: expresión normalizada + log1p,
  - `var["highly_variable"]`: etiqueta booleana para genes HVG.
- Este archivo es la entrada estándar para los notebooks de PCA, Harmony, vecinos y UMAP.

In [10]:
data_processed_dir = CONFIG["DATA_PROCESSED_DIR"]

# Guardamos solo el objeto completo normalizado, que ya incluye
# la anotación de genes altamente variables en adata.var["highly_variable"].
normalized_path = data_processed_dir / "TFM_CIRRHOSIS_normalized.h5ad"

adata.write_h5ad(normalized_path)

print(f"\nObjeto normalizado con anotación de HVG guardado en:\n{normalized_path}")


Objeto normalizado con anotación de HVG guardado en:
D:\Users\Coni\Documents\TFM_CirrhosIS\data_processed\TFM_CIRRHOSIS_normalized.h5ad


## 9. Resumen de este notebook

- Entrada: `TFM_CIRRHOSIS_afterQC.h5ad` (células ya filtradas por QC).
- Se guarda una copia de las cuentas crudas en `layers["counts"]`.
- Se aplica normalización a 10k counts por célula + `log1p`, guardando el resultado en:
  - `adata.X` y `layers["log1p_10k"]`.
- Se seleccionan ~3000 HVG con `highly_variable_genes` sobre `layer="counts"`,
  considerando `batch_key` cuando procede y excluyendo MT/RPL/RPS/HB/HLA.
- Se escribe `TFM_CIRRHOSIS_normalized.h5ad`, que contiene todo lo necesario
  para la integración (Harmony) y la reducción de dimensionalidad posteriores.

## Comentarios

- En la ejecución real del TFM:
  - Dimensiones de entrada tras QC: ~225 944 células × 38 606 genes.
  - `batch_key` detectado: `gem_id` con 8 bibliotecas.
  - Se marcan inicialmente 3000 HVG y, tras excluir MT/RPL/RPS/HB/HLA,
    quedan ~2976 genes altamente variables.
- El archivo `TFM_CIRRHOSIS_normalized.h5ad` conserva:
  - capas coherentes (`counts`, `log1p_10k`),
  - anotación de HVG y batch,
  y es el punto de partida previsto para el siguiente notebook de integración con Harmony.